# **Custom Models and Training with TensorFlow**
Up until now, we’ve used only TensorFlow’s high-level API, Keras, but it already got us pretty far: we built various neural network architectures, including regression and classification nets, Wide & Deep nets, and self-normalizing nets, using all sorts of techniques, such as batch normalization, dropout, and learning rate schedules. In fact, 95% of the use cases you will encounter will not require anything other than Keras (and tf.data; see Chapter 13). But now it’s time to dive deeper into TensorFlow and take a look at its lower-level *Python API(htts://homl.info/tf2api)*. This will be useful when you need extra control to write custom loss functions, custom metrics, layers, models, initializers, regularizers, weight constraints, and more. You may even need to fully control the training loop itself; for example, to apply special transformations or constraints to the gradients (beyond just clipping them) or to use multiple optimizers for different parts of the network. We will cover all these cases in this chapter, and we will also look at how you can boost your custom models and training algorithms using TensorFlow’s automatic graph generation feature. But first, let’s take a quick tour of TensorFlow.

## **A Quick Tour of TensorFlow**
As you know, TensorFlow is a powerful library for numerical computation, particularly well suited and fine-tuned for large-scale machine learning (but you can use it for anything else that requires heavy computations). It was developed by the Google Brain team and it powers many of Google’s largescale services, such as Google Cloud Speech, Google Photos, and Google Search. It was open sourced in November 2015, and it is now the most widely used deep learning library in the industry: countless projects use TensorFlow for all sorts of machine learning tasks, such as image classification, natural language processing, recommender systems, and time series forecasting.

So what does TensorFlow offer? Here's a summary:
- Its core is very similar to Numpy, but with GPU support

- It supports distributed computing (across multiple devices and servers).
  
- It includes a kind of just-in-time (**JIT**) compiler that allows it to optimize computations for speed and memory usage. It works by extracting the ***computation graph*** from a Python function, optimizing it (e.g., by pruning unused nodes), and running it efficiently (e.g., by automatically running independent operations in parallel).
  
- Computation graphs can be exported to a portable format, so you can train a TensorFlow model in one environment (e.g., using Python on Linux) and run it in another (e.g., using Java on an Android device).

-  It implements reverse-mode autodiff (see Chapter 10 and Appendix B) and provides some excellent optimizers, such as RMSProp and Nadam (see Chapter 11), so you can easily minimize all sorts of loss functions.

TensorFlow offers many more features built on top of these core features: the most important is of course Keras,  but it also has data loading and preprocessing ops (tf.data, tf.io, etc.), image processing ops (tf.image), signal processing ops (tf.signal), and more

> #### **TIP**
> We will cover many of the packages and functions of the TensorFlow API, but it’s impossible to cover them all, so you should really take some time to browse through the API; you will find that it is quite rich and well documented.

At the lowest level, each TensorFlow operation (op for short) is implemented using highly efficient C++ code.  Many operations have multiple implementations called kernels: each kernel is dedicated to a specific device type, such as **CPUs**, **GPUs**, or even **TPUs** (***tensor processing units**). As you may know, **GPUs** can dramatically speed up computations by splitting them into many smaller chunks and running them in parallel across many **GPU** threads. **TPUs** are even faster: they are custom **ASIC** chips built specifically for deep learning operations  (we will discuss how to use TensorFlow with **GPUs** or **TPUs** in Chapter 19).

Most of the time your code will use the high-level APIs (especially Keras and tf.data), but when you need more flexibility you will use the lower-level Python API, handling tensors directly. In any case, TensorFlow’s execution engine will take careof running the operations efficiently, even across multiple devices and machines if you tell it to.

TensorFlow runs not only on Windows, Linux, and macOS, but also on mobile devices (using ***TensorFlow Lite***), including both **iOS** and **Android** (see Chapter 19). Note that **APIs** for other languages are also available, if you do not want to use the Python **API**: there are C++, Java, and Swift APIs. There is even a JavaScript implementation called ***TensorFlow.js*** that makes it possible to run your models directly in your browser.

There’s more to TensorFlow than the library. TensorFlow is at the center of an extensive ecosystem of libraries. First, there’s TensorBoard for visualization (see Chapter 10). Next, there’s [*TensorFlow Extended* (**TFX**), https://tensorflow.org/tfx], which is a set of libraries built by Google to productionize TensorFlow projects: it includes tools for data validation, preprocessing, model analysis, and serving (with TF Serving; see Chapter 19). Google’s TensorFlow Hub provides a way to easily download and reuse pretrained neural networks. You can also get many neural network architectures, some of them pretrained, in TensorFlow’s *model garden(https://tensorflow.org/resources)*. Check out the TensorFlow Resources and *https://github.com/jtoy/awesome-tensorflow* for moreTensorFlow-based projects. You will find hundreds of TensorFlow projects on GitHub, so it is often easy to find existing code for whatever you are trying to do.

> #### **TIP**
> More and more **ML** papers are released along with their implementations, and sometimes even with pretrained models. Check out *https://paperswithcode.com* to easily find them.

Last but not least, TensorFlow has a dedicated team of passionate and helpful developers, as well as a large community contributing to improving it. To ask technical questions, you should use *https://stackoverflow.com* and tag your question with ***tensorflow*** and ***python***. You can file bugs and feature requests through **GitHub**(*https://discuss.tensorflow.org*). For general discussions, join the **TensorFlow Forum** '*https://discuss.tensorflow.org*'. 

OK, it’s time to start coding!

## **Using TensorFlow like Numpy**
TensorFlow’s API revolves around ***tensors***, which flow from operation to operation—hence the name ***TensorFlow***. A tensor is very similar to a NumPy ***ndarray***: it is usually a multidimensional array, but it can also hold a scalar (a simple value, such as ***42***). These tensors will be important when we create custom cost functions, custom metrics, custom layers, and more, so let’s see how to create and manipulate them.

### **Tensors and Operations**
You can create a tensor with ***tf.constant()***. For example, here is a tensor representing a matrix with two rows and three columns of floats:

In [1]:
import tensorflow as tf

In [ ]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]]) # Matrix
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

Just like an ***ndarray***, a ***tf.Tensor*** has a shape and a data type (***dtype***):

In [8]:
print(t.shape)
print(t.dtype)

(2, 3)
<dtype: 'float32'>


Indexing works much like in NumPy:

In [9]:
print(t[:, 1:])
print(t[:, 1, tf.newaxis])

tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[2.]
 [5.]], shape=(2, 1), dtype=float32)


Most importantly, all sorts of tensor operations are available:

In [10]:
print(t + 10)
print(tf.add(t, 10))
print('\n')

print(tf.square(t))
print('\n')

print(t @ tf.transpose(t))
print(tf.matmul(t, tf.transpose(t)))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)


tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)


tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


Note that writing ***t + 10*** is equivalent to calling ***tf.add(t, 10)*** (indeed, Python calls the magic method ***t.__add __(10)***, which just calls ***tf.add(t, 10))***. Other operators, like - and, are also supported.The ***@*** operator was added in Python 3.5, for matrix multiplication: it is equivalent to calling the ***tf.matmul()*** function.

> #### **NOTE**
> Many functions and classes have aliases. For example, ***tf.add()*** and ***tf.math.add()*** are the same function. This allows TensorFlow to have concise names for the most common operations  while preserving well-organized packages.

A tensor can also hold a scalar value. In this case, the shape is empty:

In [ ]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

> #### **NOTE**
> The Keras API has its own low-level API, located in ***tf.keras.backend***. This package is usually imported as ***K***, for conciseness. It used to include functions like ***K.square()***, ***K.exp()***, and ***K.sqrt()***, which you may run across in existing code: this was useful to write portable code back when Keras supported multiple backends, but now that Keras is TensorFlow-only, you should call TensorFlow’s lowlevel API directly (e.g., ***tf.square()*** instead of ***K.square()***). Technically ***K.square()*** and its friends are still there for backward compatibility, but the documentation of the ***tf.keras.backend*** package only lists a handful of utility functions, such as ***clear_session()*** (mentioned in Chapter 10).

You will find all the basic math operations you need (***tf.add()***, ***tf.multiply()***, ***tf.square()***, ***tf.exp()***, ***tf.sqrt()***, etc.) and most operations that you can find in NumPy (e.g., ***tf.reshape()***, ***tf.squeeze()***, ***tf.tile()***). Some functions have a different name than in NumPy; for instance, ***tf.reduce_mean()***, ***tf.reduce_sum()***, ***tf.reduce_max()***, and ***tf.math.log()*** are the equivalent of ***np.mean()***, ***np.sum()***, ***np.max()***, and ***np.log()***. When the name differs, there is usually a good reason for it. For example, inTensorFlow you must write ***tf.transpose(t***); you cannot just write ***t.T*** like in NumPy. The reason is that the ***tf.transpose()*** function does not do exactly the same thing as NumPy’s **T** attribute: in TensorFlow, a new tensor is created with its own copy of the transposed data, while in NumPy, ***t.T*** is just a transposed view on the same data. Similarly, the ***tf.reduce_sum()*** operation is named this way because its **GPU** kernel (i.e., GPU implementation) uses a reduce algorithm that does not guarantee the order in which the elements are added: because 32-bit floats have limited precision, the result may change ever so slightly every time you call this operation. The same is true of ***tf.reduce_mean()*** (but of course ***tf.reduce_max()*** is deterministic).

### **Tensors and Numpy**
Tensors play nice with NumPy: you can create a tensor from a NumPy array, and vice versa. You can even apply TensorFlow operations to NumPy arrays and NumPy operations to tensors:

In [11]:
import numpy as np

a = np.array([2., 4., 5.])
print(tf.constant(a))
print('\n')

print(t.numpy()) # or np.array(t)
print('\n')

print(tf.square(a))
print('\n')

print(np.square(t))

tf.Tensor([2. 4. 5.], shape=(3,), dtype=float64)


[[1. 2. 3.]
 [4. 5. 6.]]


tf.Tensor([ 4. 16. 25.], shape=(3,), dtype=float64)


[[ 1.  4.  9.]
 [16. 25. 36.]]


> #### **WARNING**
> Notice that NumPy uses 64-bit precision by default, while TensorFlow uses 32-bit. This is because 32-bit precision is generally more than enough for neural networks, plus it runs faster and uses less RAM. So when you create a tensor from a NumPy array, make sure to set ***dtype=tf.float32***.

### **Type Conversions**
Type conversions can significantly hurt performance, and they can easily go unnoticed when they are done automatically. To avoid this, TensorFlow does not perform any type conversions automatically: it just raises an exception if you try to execute an operation on tensors with incompatible types. For example, you cannot add a float tensor and an integer tensor, and you cannot even add a 32-bit float and a 64-bit float:

In [45]:
tf.constant(2.) + tf.constant(40)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: 

In [ ]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2] name: 

This may be a bit annoying at first, but remember that it’s for a good cause! And of course you can use ***tf.cast()*** when you really need to convert types:

In [12]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### **Variables**
The ***tf.Tensor*** values we’ve seen so far are immutable: we cannot modify them. This means that we cannot use regular tensors to implement weights in a neural network, since they need to be tweaked by backpropagation. Plus, other parameters may also need to change over time(e.g., a momentum optimizer keeps track of past gradients). What we need is a ***tf.Variable***:

In [13]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

A ***tf.Variable*** acts much like a ***tf.Tensor***: you can perform the same operations with it, it plays nicely with NumPy as well, and it is just as picky with types. But it can also be modified in place using the ***assign()*** method (or ***assign_add()*** or ***assign_sub()***, which increment or decrement the variable by the given value). You can also modify individual cells (or slices), by using the cell’s (or slice’s) ***assign()*** method or by using the ***scatter_update()*** or ***scatter_nd_update()*** methods:

In [14]:
print(v.assign(2 * v)) # v now equals [[2., 4., 6.], [8., 10., 12.]]
print('\n')

print(v[0, 1].assign(42)) # v now equals [[2., 42., 6.], [8., 10., 12.]]
print('\n')

print(v[:, 2].assign([0., 1.])) # v now equals [[2., 42., 0.], [8., 10., 1.]]
print('\n')

print(v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])) # v now equals [[100., 42., 0.], [8., 10., 200.]]

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>


<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>


Direct assignment will not work:

In [ ]:
v[1] = [7., 8., 9.]

TypeError: 'ResourceVariable' object does not support item assignment

> #### **NOTE**
>  In practice you will rarely have to create variables manually; Keras provides an ***add_weight()*** method that will take care of it for you, as you’ll see. Moreover, model parameters will generally be updated directly by the optimizers, so you will rarely need to update variables manually.


### **Other Data Structures**
TensorFlow supports several other data structures, including the following (see the “Other Data Structures” section in this chapter’s notebook or Appendix C for more details):
- ***Sparse tensors***(*tf.SparseTensor*)
    
    Efficiently represent tensors containing mostly zeros. The tf.sparse package contains operations for sparse tensors.

- ***Tensor arrays*** (*tf.TensorArray*)
    
    Are lists of tensors. They have a fixed length by default but can optionally be made extensible. All tensors they contain must have the same shape and data type.

-  ***Ragged tensors*** (*tf.RaggedTensor*)

    Represent lists of tensors, all of the same rank and data type, but with varying sizes. The dimensions along which the tensor sizes vary are called the ragged dimensions. The *tf.ragged* package contains operations for ragged tensors

-  ***String tensors***
  
    Are regular tensors of type *tf.string*. These represent byte strings, not Unicode strings, so if you create a string tensor using a Unicode string (e.g., a regular Python 3 string like "*café*"), then it will get encoded to UTF-8 automatically (e.g., *b"caf\xc3\xa9*"). Alternatively, you can represent Unicode strings using tensors of type *tf.int32*, where each item represents a Unicode code point (e.g., *[99, 97, 102, 233]*). The *tf.strings* package (with an s) contains ops for byte strings and Unicode strings (and to convert one into the other). It’s important to note that a *tf.string* is atomic, meaning that its length does not appear in the tensor’s shape. Once you convert it to a Unicode tensor (i.e., a tensor of type *tf.int32* holding Unicode code points), the length appears in the shape.

- ***Sets***
  
    Are represented as regular tensors (or sparse tensors). For example, *tf.constant([[1, 2], [3, 4]])* represents the two sets {1, 2} and {3, 4}. More generally, each set is represented by a vector in the tensor’s last axis. You can manipulate sets using operations from the *tf.sets* package.

- ***Queues***
    Store tensors across multiple steps. TensorFlow offers various kinds of queues: basic first-in, first-out (**FIFO**) queues (*FIFOQueue*), plus queues that can prioritize some items (*PriorityQueue*), shuffle their items (*RandomShuffleQueue*), and batch items of different shapes by padding (*PaddingFIFOQueue*). These classes are all in the *tf.queue* package.

With tensors, operations, variables, and various data structures at your disposal, you are now ready to customize your models and training algorithms!

## **Customizing Models and Training Algorithms**
You’ll start by creating a custom loss function, which is a straightforward and common use case.

### **Custom Loss Functions**

Suppose you want to train a regression model, but your training set is a bit noisy. Of course, you start by trying to clean up your dataset by removing or fixing the outliers, but that turns out to be insufficient; the dataset is still noisy. Which loss function should you use? The mean squared error might penalize large errors too much and cause your model to be imprecise. The mean absolute error would not penalize outliers as much, but training might take a while to converge, and the trained model might not be very precise.

This is probably a good time to use the Huber loss (introduced in Chapter 10) instead of the good old MSE. The Huber loss is available in Keras (just use an instance of the tf.keras.losses.Huber class), but let’s pretend it’s not there. To implement it, just create a function that takes the labels and the model’s predictions as arguments, and uses TensorFlow operations to compute a tensor containing all the losses (one per sample):

In [15]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

> #### **WARNING**
>  For better performance, you should use a vectorized implementation, as in this example. Moreover, if you want to benefit from TensorFlow’s graph optimization features, you should use only TensorFlow operations.

It is also possible to return the mean loss instead of the individual sample losses, but this is not recommended as it makes it impossible to use class weights or sample weights when you need them (see Chapter 10).

Now you can use this Huber loss function when you compile the Keras model, then train your model as usual:

And that’s it! For each batch during training, Keras will call the ***huber_fn()*** function to compute the loss, then it will use reverse-mode autodiff to compute the gradients of the loss with regard to all the model parameters, and finally it will perform a gradient descent step (in this example using a Nadam optimizer). Moreover, it will keep track of the total loss since the beginning of the epoch, and it will display the mean loss.

But what happens to this custom loss when you save the model?

### **Saving and Loading Models That Contain Custom Components**
Saving a model containing a custom loss function works fine, but when you load it, you’ll need to provide a dictionary that maps the function name to the actual function. More generally, when you load a model containing custom objects, you need to map the names to the objects:

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(x_train_full, y_train_full), (x_test, y_test) = fashion_mnist
x_train, y_train = x_train_full[:-5000], y_train_full[:-5000]
x_valid, y_valid = x_train_full[-5000:], y_train_full[-5000:]

x_train, x_valid, x_test = x_train / 255., x_valid / 255., x_test / 255.


In [ ]:

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

tf.random.set_seed(50)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]), 
    tf.keras.layers.Dense(400, activation='relu'), 
    tf.keras.layers.Dense(300, activation='relu'), 
    tf.keras.layers.Dense(200, activation='relu'), 
    tf.keras.layers.Dense(100, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='sgd', 
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=18, 
                    validation_data=(x_valid, y_valid))

filepath = "D:/CONTENTS/AI/CODE/Neural Networks and Deep Learning/Models/model.keras"
model.save(filepath=filepath)

In [18]:
model_A = tf.keras.models.load_model(filepath)
model = tf.keras.models.load_model(filepath, custom_objects={filepath : huber_fn})

> #### **TIP**
> If you decorate the ***huber_fn()*** function with ***@keras.utils.register_keras_serializable()***, it will automatically be available to the ***load_model()*** function: there’s no need to include it in the ***custom_objects*** dictionary.

With the current implementation, any error between –1 and 1 is considered “small”. But what if you want a different threshold? One solution is to create a function that creates a configured loss function:

In [19]:
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.squared(error) / 2
        linear_loss = threshold * tf.abs(error) - threshold ** 2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

model.compile(loss=create_huber(2.0), optimizer='nadam')

Unfortunately, when you save the model, the ***threshold*** will not be saved. This means that you will have to specify the ***threshold*** value when loading the model (note that the name to use is "***huber_fn***", which is the name of the function you gave Keras, not the name of the function that created it):

In [21]:
model = tf.keras.models.load_model(
    filepath, 
    custom_objects={'huber_fn' : create_huber(2.0)}
)

You can solve this by creating a subclass of the ***tf.keras.losses.Loss*** class, and then implementing its ***get_config()*** method:

In [22]:
class HuberLoss(tf.keras.losses.Loss):
    def __init__(self, threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
        
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

Let’s walk through this code:
- The constructor accepts *****kwargs*** and passes them to the parent constructor, which handles standard hyperparameters: the ***name*** of the loss and the ***reduction*** algorithm to use to aggregate the individual instance losses. By default this is "***AUTO***", which is equivalent to "***SUM_OVER_BATCH_SIZE***": the loss will be the sum of the instance losses, weighted by the sample weights, if any, and divided by the batch size (not by the sum of weights, so this is not the weighted mean).  Other possible values are "***SUM***" and "***NONE***".

- The ***call()*** method takes the labels and predictions, computes all the instance losses, and returns them. 
  
- The ***get_config()*** method returns a dictionary mapping each hyperparameter name to its value. It first calls the parent class’s ***get_config()*** method, then adds the new hyperparameters to this dictionary. 
  
You can then use any instance of this class when you compile the model:

In [23]:
model.compile(loss=HuberLoss(2.), optimizer='nadam')

When you save the model, the threshold will be saved along with it; and when you load the model, you just need to map the class name to the class itself:

In [25]:
model = tf.keras.models.load_model(filepath, 
                                   custom_objects={'HuberLoss':HuberLoss})

When you save a model, Keras calls the loss instance’s ***get_config()*** method and saves the config in the SavedModel format. When you load the model, it calls the ***from_config()*** class method on the ***HuberLoss*** class: this method is implemented by the base class (***Loss***) and creates an instance of the class, passing *****config*** to the constructor.

That’s it for losses! As you’ll see now, custom activation functions, initializers, regularizers, and constraints are not much different.

### **Custom Activation Functions, Initializers, Regularizers, and Constraints**

Most Keras functionalities, such as losses, regularizers, constraints, initializers, metrics, activation functions, layers, and even full models, can be customized in much the same way. Most of the time, you will just need to write a simple function with the appropriate inputs and outputs. Here are examples of a custom activation function (equivalent to ***tf.keras.activations.softplus()*** or ***tf.nn.softplus()***), a custom Glorot initializer (equivalent to ***tf.keras.initializers.glorot_normal()***), a custom ℓ regularizer (equivalent to ***tf.keras.regularizers.l1(0.01)***), and a custom constraint that ensures weights are all positive (equivalent to ***tf.keras.constraints.nonneg()*** or ***tf.nn.relu()***):

In [26]:
def my_softplus(z):
    return tf.math.log(1.0 + tf.exp(z))

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_likes(weights), weights)

As you can see, the arguments depends on the tpe of the custom function. These custom functions can be used normally, as shown here:

In [27]:
layer = tf.keras.layers.Dense(1, activation=my_softplus, 
                              kernel_initializer=my_glorot_initializer, 
                              kernel_regularizer=my_l1_regularizer, 
                              kernel_constraint=my_positive_weights)

The activation function will be applied to the output of this ***Dense*** layer, and its result will be passed on to the next layer. The layer’s weights will be initialized using the value returned by the initializer. At each training step the weights will be passed to the regularization function to compute the regularization loss, which will be added to the main loss to get the final loss used for training. Finally, the constraint function will be called after each training step, and the layer’s weights will be replaced by the constrained weights.

If a function has hyperparameters that need to be saved along with the model, then you will want to subclass the appropriate class, such as ***tf.keras.regularizers.Regularizer***, ***tf.keras.constraints.Constraint***, ***tf.keras.initializers.Initializer***, or ***tf.keras.layers.Layer*** (for any layer, including activation functions). Much as you did for the custom loss, here is a simple class for ℓ<sub>1</sub> regularization that saves its ***factor*** hyperparameter (this time you do not need to call the parent constructor or the ***get_config()*** method, as they are not defined by the parent class):

In [28]:
class MyRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
        
    def __cal__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    
    def get_config(self):
        return {'factor': self.factor}

Note that you must implement the ***call()*** method for losses, layers (including activation functions), and models, or the ***__call __()*** method for regularizers, initializers, and constraints. For metrics, things are a bit different, as you will see now.


### **Custom Metrics**

Losses and metrics are conceptually not the same thing: losses (e.g., cross entropy) are used by gradient descent to train a model, so they must be differentiable (at least at the points where they are evaluated), and their gradients should not be zero everywhere. Plus, it’s OK if they are not easily interpretable by humans. In contrast, metrics (e.g., accuracy) are used to evaluate a model: they must be more easily interpretable, and they can be nondifferentiable or have zero gradients everywhere.

That said, in most cases, defining a custom metric function is exactly the same as defining a custom loss function. In fact, we could even use the Huber loss function we created earlier as a metric;  it would work just fine (and persistence would also work the same way, in this case only saving the name of the function, "***huber_fn***", not the threshold):

In [29]:
model.compile(loss='mse', optimizer='nadam', metrics=[create_huber(2.0)])

For each batch during training, Keras will compute this metric and keep track of its mean since the beginning of the epoch. Most of the time, this is exactly what you want. But not always! Consider a binary classifier’s precision, for example. As you saw in Chapter 3, precision is the number of true positives divided by the number of positive predictions (including both true positives and false positives). Suppose the model made five positive predictions in the first batch, four of which were correct: that’s 80% precision. Then suppose the model made three positive predictions in the second batch, but they were all incorrect: that’s 0% precision for the second batch. If you just compute the mean of these two precisions, you get 40%. But wait a second—that’s not the model’s precision over these two batches! Indeed, there were a total of four true positives (4 + 0) out of eight positive predictions (5 + 3), so the overall precision is 50%, not 40%. What we need is an object that can keep track of the number of true positives and the number of false positives and that can compute the precision based on these numbers when requested. This is precisely what the ***tf.keras.metrics.Precision*** class does:

In [30]:
precision = tf.keras.metrics.Precision()
print(precision([0, 1, 1, 1, 0, 1, 0, 1], [1, 1, 0, 1, 0, 1, 0, 1]))
print(precision([0, 1, 0, 0, 1, 0, 1, 1], [1, 0, 1, 1, 0, 0, 0, 0]))

tf.Tensor(0.8, shape=(), dtype=float32)
tf.Tensor(0.5, shape=(), dtype=float32)


In this example, we created a ***Precision*** object, then we used it like a function, passing it the labels and predictions for the first batch, then for the second batch (you can optionally pass sample weights as well, if you want). We used the same number of true and false positives as in the example we just discussed. After the first batch, it returns a precision of 80%; then after the second batch, it returns 50% (which is the overall precision so far, not the second batch’s precision). This is called a ***streaming metric*** (or ***stateful metric***), as it is gradually updated, batch after batch.

At any point, we can call the ***result()*** method to get the current value of the metric. We can also look at its variables (tracking the number of true and false positives) by using the ***variables*** attribute, and we can reset these variables using the ***reset_state()*** method:

In [31]:
print(precision.result())
print('\n')
print(precision.variables)
print('\n')
precision.reset_state() # both variables get reset to 0.0

tf.Tensor(0.5, shape=(), dtype=float32)


[<Variable path=precision/true_positives, shape=(1,), dtype=float32, value=[4.]>, <Variable path=precision/false_positives, shape=(1,), dtype=float32, value=[4.]>]




If you need to define your own custom streaming metric, create a subclass of the ***tf.keras.metrics.Metric*** class. Here is a basic example that keeps track of the total Huber loss and the number of instances seen so far. When asked for the result, it returns the ratio, which is just the mean Huber loss:

In [32]:
class HuberMetric(tf.keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # Handles base args (e.g., dtype)
        self.threshold = threshold
        self.huber_fn = Create_huber(threshold)
        self.total = self.add_weight('total', initializer='zeros')
        self.count = self.add_weight('count', initializer='zeros')
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        sample_metrics = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(sample_metrics))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
        
    def result(self):
        return self.total / self.count
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold}

Let’s walk through this code:
- The constructor uses the ***add_weight()*** method to create the variables needed to keep track of the metric’s state over multiple batches—in this case, the sum of all Huber losses (***total***) and the number of instances seen so far (***count***). You could just create variables manually if you preferred. Keras tracks any ***tf.Variable*** that is set as an attribute (and more generally, any “trackable” object, such as layers or models).
  
- The ***update_state()*** method is called when you use an instance of this class as a function (as we did with the ***Precision*** object). It updates the variables, given the labels and predictions for one batch (and sample weights, but in this case we ignore them).

- The ***result()*** method computes and returns the final result, in this case the mean Huber metric over all instances. When you use the metric as a function, the ***update_state()*** method gets called first, then the ***result()*** method is called, and its output is returned.

- We also implement the ***get_config()*** method to ensure the ***threshold*** gets saved along with the model.

- The default implementation of the ***reset_state()*** method resets all variables to 0.0 (but you can override it if needed).

> #### **NOTE**
>  Keras will take care of variable persistence seamlessly; no action is required.

When you define a metric using a simple function, Keras automatically calls it for each batch, and it keeps track of the mean during each epoch, just like we did manually. So the only benefit of our ***HuberMetric*** class is that the ***threshold*** will be saved. But of course, some metrics, like precision, cannot simply be averaged over batches: in those cases, there’s no other option than to implement a streaming metric.

Now that you’ve built a streaming metric, building a custom layer will seem like a walk in the park!

### **Custom Layers**
You may occasionally want to build an architecture that contains an exotic layer for which TensorFlow does not provide a default implementation. Or you may simply want to build a very repetitive architecture, in which a particular block of layers is repeated many times, and it would be convenient to treat each block as a single layer. For such cases, you’ll want to build a custom layer.

There are some layers that have no weights, such as ***tf.keras.layers.Flatten*** or ***tf.keras.layers.ReLU***. If you want to create a custom layer without any weights, the simplest option is to write a function and wrap it in a ***tf.keras.layers.Lambda*** layer. For example, the following layer will apply the exponential function to its inputs:

In [33]:
exponential_layer = tf.keras.layers.Lambda(lambda x: tf.exp(x))

This custom layer can then be used like any other layer, using the sequential **API**, the functional **API**, or the subclassing **API**. You can also use it as an activation function, or you could use ***activation=tf.exp***. The exponential layer is sometimes used in the output layer of a regression model when the values to predict have very different scales (e.g., 0.001, 10., 1,000.). In fact, the exponential function is one of the standard activation functions in Keras, so you can just use ***activation="exponential"***.

As you might guess, to build a custom stateful layer (i.e., a layer with weights), you need to create a subclass of the ***tf.keras.layers.Layer*** class. For example, the following class implements a simplified version of the Dense layer:

In [34]:
class MyDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = tf.keras.activation.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name='kernel', shape=[batch_input_shape[-1], 
                                  self.units], 
            initializer='glorot_normal'
        )
        self.bias = self.add_weight(
            name='bias', shape=[self.units], initializer='zeros'
        )
        
    def call(self, x):
        return self.activation(x @ self.kernel + self.bias)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units': self.units, 'activation': 
            tf.keras.activations.serializer(self.activation)}
        

Let's walk through this code:
- The constructor takes all the hyperparameters as arguments (in this example, ***units*** and ***activation***), and importantly it also takes a *****kwargs*** argument. It calls the parent constructor, passing it the ***kwargs***: this takes care of standard arguments such as ***input_shape, trainable***, and ***name***. Then it saves the hyperparameters as attributes, converting the ***activation*** argument to the appropriate activation function using the ***tf.keras.activations.get()*** function (it accepts functions, standard strings like "***relu***" or "***swish***", or simply None).

- The ***build()*** method’s role is to create the layer’s variables by calling the ***add_weight()*** method for each weight. The ***build()*** method is called the first time the layer is used. At that point, Keras will know the shape of this layer’s inputs, and it will pass it to the ***build()*** method,  which is often necessary to create some of the weights. For example, we need to know the number of neurons in the previous layer in order to create the connection weights matrix (i.e., the "***kernel***"): this corresponds to the size of the last dimension of the inputs. At the end of the ***build()*** method (and only at the end), you must call the parent’s ***build()*** method: this tells Keras that the layer is built (it just sets ***self.built = True***).

- The ***call()*** method performs the desired operations. In this case, we compute the matrix multiplication of the inputs X and the layer’s kernel, we add the bias vector, and we apply the activation function to the result, and this gives us the output of the layer.

- The ***get_config()*** method is just like in the previous custom classes. Note that we save the activation function’s full configuration by calling ***tf.keras.activations.serialize()***.

You can now use a MyDense layer just like any other layer!

> #### **NOTE**
>  Keras automatically infers the output shape, except when the layer is dynamic (as you will see shortly). In this (rare) case, you need to implement the ***compute_output_shape()*** method, which must return a ***TensorShape*** object.

To create a layer with multiple inputs (e.g., ***Concatenate***), the argument to the ***call()*** method should be a tuple containing all the inputs. To create a layer with multiple outputs, the ***call()*** method should return the list of outputs. For example, the following toy layer takes two inputs and returns three outputs:

In [35]:
class MyMultilayer(tf.keras.layers.Layer):
    def call(self, x):
        x1, x2 = x
        return x1 + x2, x1 * x2, x1 / x2

This layer may now be used like any other layer, but of course only using the functional and subclassing **APIs**, not the sequential **API** (which only accepts layers with one input and one output).

If your layer needs to have a different behavior during training and during testing (e.g., if it uses ***Dropout*** or ***BatchNormalization*** layers), then you must add a ***training*** argument to the ***call()*** method and use this argument to decide what to do. For example, let’s create a layer that adds Gaussian noise during training (for regularization) but does nothing during testing (Keras has a layer that does the same thing, ***tf.keras.layers.GaussianNoise***):

In [36]:
class MyGaussianNoise(tf.keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev
        
    def call(self, x, training=False):
        if training:
            noise = tf.rarandom.normal(tf.shape(x),
                                       stddev=self.stddev)
            return x + noise
        else:
            return x

With that, you can now build any custom layer you need! Now let’s look at how to create custom models.

### **Custom Models**
We already looked at creating custom model classes in Chapter 10, when we discussed the subclassing API.  It’s straightforward: subclass the ***tf.keras.Model*** class, create layers and variables in the constructor, and implement the ***call()*** method to do whatever you want the model to do.

Consider this model ilustration, The inputs go through a first dense layer, then through a *residual block* composed of two dense layers and an addition operation (as you will see in Chapter 14, a residual block adds its inputs to its outputs), then through this same residual block three more times, then through a second residual block, and the final result goes through a dense output layer. Don’t worry if this model does not make much sense; it’s just an example to illustrate the factthat you can easily build any kind of model you want, even one that contains loops and skip connections. To implement this model, it is best to first create a ***ResidualBlock*** layer, since we are going to create a couple of identical blocks (and we might want to reuse it in another model):

In [37]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(n_neurons, 
                                             activation='relu', 
                                             kernel_initializer='he_normal')
                       for _ in range(n_layers)]
        
    def call(self, inputs):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        return inputs + z
    

This layer is a bit special since it contains other layers. This is handled transparently by Keras: it automatically detects that the ***hidden*** attribute contains trackable objects (layers in this case), so their variables are automatically added to this layer’s list of variables. The rest of this class is self-explanatory. Next, let’s use the subclassing API to define the model itself:

In [38]:
class ResidualRegressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = tf.keras.layers.Dense(30, activation='relu', 
                                             kernel_initializer='he_normal')
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = tf.keras.layers.Dense(output_dim)
        
    def call(self, inputs):
        z = self.hidden1(inputs)
        for _ in range(1 + 3):
            z = self.block1(z)
            z = self.block2(z)
            return self.out(z)
        

We create the layers in the constructor and use them in the ***call()*** method. This model can then be used like any other model (compile it, fit it, evaluate it, and use it to make predictions). If you also want to be able to save the model using the ***save()*** method and load it using the ***tf.keras.models.load_model()*** function, you must implement the ***get_config()*** method (as we did earlier) in both the ***ResidualBlock*** class and the ***ResidualRegressor*** class. Alternatively, you can save and load the weights using the ***save_weights()*** and ***load_weights()*** methods.

The ***Model*** class is a subclass of the ***Layer*** class, so models can be defined and used exactly like layers. But a model has some extra functionalities, including of course its ***compile()***, ***fit()***, ***evaluate()***, and ***predict()*** methods (and a few variants), plus the ***get_layer()*** method (which can return any of the model’s layers by name or by index) and the ***save()*** method (and support for ***tf.keras.models.load_model()*** and ***tf.keras.models.clone_model()***).

> #### **TIP**
> If models provide more functionality than layers, why not just define every layer as a model? Well, technically you could, but it is usually cleaner to distinguish the internal components of your model (i.e., layers or reusable blocks of layers) from the model itself (i.e., the object you will train). The former should subclass the ***Layer*** class, while the latter should subclass the ***Model*** class.

With that, you can naturally and concisely build almost any model that you find in a paper, using the sequential API, the functional API, the subclassing API, or even a mix of these. “Almost” any model? Yes, there are still a few things that we need to look at: first, how to define losses or metrics based on model internals, and second, how to build a custom training loop.

### **Losses ans Metrics Based on Model Internals**
The custom losses and metrics we defined earlier were all based on the labels and the predictions (and optionally sample weights). There will be times when you want to define losses based on other parts of your model, such as the weights or activations of its hidden layers. This may be useful for regularization purposes or to monitor some internal aspect of your model.

To define a custom loss based on model internals, compute it based on any part of the model you want, then pass the result to the ***add_loss()*** method. For example, let’s build a custom regression MLP model composed of a stack of five hidden layers plus an output layer. This custom model will also have an auxiliary output on top of the upper hidden layer. The loss associated with this auxiliary output will be called the *reconstruction loss* (see Chapter 17): it is the mean squared difference between the reconstruction and the inputs. By adding this reconstruction loss to the main loss, we will encourage the model to preserve as much information as possible through the hidden layers—even information that is not directly useful for the regression task itself. In practice, this loss sometimes improves generalization (it is a regularization loss). It is also possible to add a custom metric using the model’s ***add_metric()*** method. Here is the code for this custom model with a custom reconstruction loss and a corresponding metric:

In [39]:
class Reconstructingregressor(tf.keras.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [tf.keras.layers.Dense(30, activation='relu', 
                                             kernel_initializer='he_normal')
                       for _ in range(5)]
        self.out = tf.keras.layers.Dense(output_dim)
        self.reconstruction_mean = tf.keras.metrics.Mean(
            name='reconstruction_error'
        )
        
    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = tf.keras.layers.Dense(n_inputs)
        
    def call(self, inputs, training=False):
        z = inputs
        for layer in self.hidden:
            z = layer(z)
        reconstruction = self.reconstruct(z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        if training:
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)
        return self.out(z)

Let's go through this code:
- The constructor creates the **DNN** with five dense hidden layers and one dense output layer. We also create a ***Mean*** streaming metric to keep track of the reconstruction error during training.

- The ***build()*** method creates an extra dense layer that will be used to reconstruct the inputs of the model. It must be created here because its number of units must be equal to the number of inputs, and this number is unknown before the ***build()*** method is called. 

- The ***call()*** method processes the inputs through all five hidden layers, then passes the result through the reconstruction layer, which produces the reconstruction.

- Then the ***call()*** method computes the reconstruction loss (the mean squared difference between the reconstruction and the inputs), and adds it to the model’s list of losses using the ***add_loss()*** method. Notice that we scale down the reconstruction loss by multiplying it by 0.05 (this is a hyperparameter you can tune). This ensures that the reconstruction loss does not dominate the main loss.

- Next, during training only, the ***call()*** method updates the reconstruction metric and adds it to the model so it can be displayed. This code example can actually be simplified by calling***self.add_metric(recon_loss)*** instead: Keras will automatically track the mean for you.

- Finally, the call() method passes the output of the hidden layers to the output layer and returns its output.

Both the total loss and the reconstruction loss will go down during training:

In most cases, everything we have discussed so far will be sufficient to implement whatever model you want to build, even with complex architectures, losses, and metrics. However, for some architectures, such as GANs (see Chapter 17), you will have to customize the training loop itself. Before we get there, we must look at how to compute gradients automatically in TensorFlow.

### **Computing Gradients Using Autodiff**
To understand how to use autodiff(see Chapter 10 and Appendix B) to compute gradients automatically, let's consider a simple toy function:

In [40]:
def f(w1, w2):
    return 3 * w1 ** 2 + 2 * w1 * w2

If you know calculus, you can analytically find that the partial derivative of this function with regard to w1 is ***6 * w1 + 2 * w2***. You can also find that its partial derivative with regard to ***w2*** is ***2 * w1***. For example, at the point ***(w1, w2) = (5, 3)***, these partial derivatives are equal to 36 and 10, respectively, so the gradient vector at this point is (36, 10). But if this were a neural network, the function would be much more complex, typically with tens of thousands of parameters, and finding the partialderivatives analytically by hand would be a virtually impossible task. One solution could be to compute an approximation of each partial derivative by measuring how much the function’s output changes when you tweak the corresponding parameter by a tiny amount:

In [41]:
w1, w2 = 5, 3
eps = 1e-6
print((f(w1 + eps, w2) - f(w1, w2)) / eps)
print('\n')
print((f(w1, w2 + eps) - f(w1, w2)) / eps)

36.000003007075065


10.000000003174137


Looks about right! This works rather well and is easy to implement, but it is just an approximation, and importantly you need to call ***f()*** at least once per parameter (not twice, since we could compute ***f(w1, w2)*** just once). Having to call ***f()*** at least once per parameter makes this approach intractable for large neural networks. So instead, we should use reverse-mode autodiff. TensorFlow makes this pretty simple:

In [42]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)
    
gradients = tape.gradient(z, [w1, w2])
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

We first define two variables ***w1*** and ***w2***, then we create a ***tf.GradientTape*** context that will automatically record every operation that involves a variable, and finally we ask this tape to compute the gradients of the result ***z*** with regard to both variables ***[w1, w2]***. Take a look at the gradients that TensorLow computed, It's Perfect! Not only is the result accurate (the precision is only limited by the floating-point errors), but the ***gradient()*** method only goes through therecorded computations once (in reverse order), no matter how many variables there are, so it is incredibly efficient. It’s like magic!

> #### **TIP**
>  In order to save memory, only put the strict minimum inside the ***tf.GradientTape() block***. Alternatively, pause recording by creating a ***with tape.stop_recording()*** block inside the ***tf.GradientTape()*** block.

The tape is automatically erased immediately after you call its ***gradient()*** method, so you will get an exception if you try to call ***gradient()*** twice:

In [ ]:
with tf.GradientTape() as tape:
    z = f(w1, w2)
    
dz_dw1 = tape.gradient(z, w1) # Returns tensor 36.0
dz_dw1
dz_dw2 = tape.gradient(z, w2) # raises a RuntimeError!

<tf.Tensor: shape=(), dtype=float32, numpy=36.0>

If you need to call ***gradient()*** more than once, you must make the tape persistent and delete it each time you are done with it to free resources: 

In [45]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)
    
dz_dw1 = tape.gradient(z, w1) # Returns tensor 36.0
dz_dw2 = tape.gradient(z, w2) # returns tensor 10.0, works fine now!
del tape

By default, the tape will only track operations involving variables, so if you try to compute the gradient of z with regard to anything other than a variable, the result will be None:

In [46]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1, c2)
    
gradients = tape.gradient(z, [c1, c2]) # returns [None, None]
gradients
    

[None, None]

However, you can force the tape to watch any tensors you like, to record every operation that involves them. You can then compute gradients with regard to these tensors, as if they were variables:

In [47]:
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)
    
gradients = tape.gradient(z, [c1, c2]) # returns [tensor 36., tensor 10.]
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

This can be useful in some cases, like if you want to implement a regularization loss that penalizes activations that vary a lot when the inputs vary little: the loss will be based on the gradient of the activations with regard to the inputs. Since the inputs are not variables, you’ll need to tell the tape to watch them.

Most of the time a gradient tape is used to compute the gradients of a single value (usually the loss) with regard to a set of values (usually the model parameters). This is where reverse-mode autodiff shines, as it just needs to do one forward pass and one reverse pass to get all the gradients at once. If you try to compute the gradients of a vector, for example a vector containing multiple losses, then TensorFlow will compute the gradients of the vector’s sum. So if you ever need to get the individual gradients (e.g., the gradients of each loss with regard to the model parameters), you must call the tape’s ***jacobian()*** method: it will perform reverse-mode autodiff once for each loss in the vector (all in parallel by default). It is even possible to compute second-order partial derivatives (the Hessians, i.e., the partial derivatives of the partial derivatives), but this is rarely needed in practice (see the “Computing Gradients Using Autodiff” section of this chapter’s notebook for an example).

In some cases you may want to stop gradients from backpropagating through some part of your neural network. To do this, you must use the ***tf.stop_gradient()*** function. The function returns its inputs during the forward pass (like ***tf.identity()***), but it does not let gradients through during backpropagation (it acts like a constant):

In [48]:
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
    z = f(w1, w2) # The foward pass is not affected by stop_gradient()
    
gradients = tape.gradient(z, [w1, w2]) # returns [tensor 30., None]
gradients    

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]

Finally, you may occasionally run into some numerical issues when computing gradients. For example, if you compute the gradients of the square root function at x = 10<sup>-50</sup> , the result will be infinite. In reality, the slope at that point is not infinite, but it’s more than 32-bit floats can handle:

In [49]:
x = tf.Variable(1e-50)
with tf.GradientTape() as tape:
    z = tf.sqrt(x)
    
tape.gradient(z, [x])

[<tf.Tensor: shape=(), dtype=float32, numpy=inf>]

To solve this, it’s often a good idea to add a tiny value to x (such as 10<sup>16</sup>) when computing its square root.

The exponential function is also a frequent source of headaches, as it grows extremely fast. For example, the way ***my_softplus()*** was defined earlier is not numerically stable. If you compute ***my_softplus(100.0)***, you will get infinity rather than the correct result (about 100). But it’s possible to rewrite the function to make it numerically stable: the softplus function is defined as log(1 + exp(z)), which is also equal to log(1 + exp(–|z|)) + max(z, 0) (see the notebook for the mathematical proof) and the advantage of this second form is that the exponential term cannot explode. So, here’s a better implementation of the ***my_softplus()*** function:

In [50]:
def my_softplus(z):
    return tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)

In some rare cases, a numerically stable function may still have numerically unstable gradients. In such cases, you will have to tell TensorFlow which equation to use for the gradients, rather than letting it use autodiff. For this, you must use the ***@tf.custom_gradient*** decorator when defining the function, and return both the function’s usual result and a function that computes the gradients. For example, let’s update the ***my_softplus()*** function to also return a numerically stable gradients function:

In [51]:
@tf.custom_gradient
def my_softplus(z):
    def my_softplus_gradients(grads): # grads = backprop'ed from upper layers
        return grads * (1 - 1 / (1 + tf.exp(z))) # stable grads of softplus
    
    result = tf.math.log(1 + tf.exp(-tf.abs(z))) + tf.maximum(0., z)
    
    return result, my_softplus_gradients
        

If you know differential calculus (see the tutorial notebook on this topic), you can find that the derivative of log(1 + exp(z)) is exp(z) / (1 + exp(z)). But this form is not stable: for large values of z, it ends up computing infinity divided by infinity, which returns NaN. However, with a bit of algebraic manipulation, you can show that it’s also equal to 1 – 1 / (1 + exp(z)), which is stable. The ***my_softplus_gradients()*** function uses this equation to compute the gradients. Note that this function will receive as input the gradients that were backpropagated so far, down to the ***my_softplus()*** function, and according to the chain rule we must multiply them with this function’s gradients.

Now when we compute the gradients of the ***my_softplus()*** function, we get the proper result, even for large input values.

Congratulations! You can now compute the gradients of any function (provided it is differentiable at the point where you compute it), evenblocking backpropagation when needed, and write your own gradient functions! This is probably more flexibility than you will ever need, even if you build your own custom training loops. You’ll see how to do that next.

### **Custom Training Loops**
In some cases, the ***fit()*** method may not be flexible enough for what you need to do. For example, the *Wide & Deep paper(https://homl.info/widedeep)* we discussed in Chapter 10 uses two different optimizers: one for the wide path and the other for the deep path. Since the ***fit()*** method only uses one optimizer (the one that we specify when compiling the model), implementing this paper requires writing your own custom loop.

You may also like to write custom training loops simply to feel more confident that they do precisely what you intend them to do (perhaps you are unsure about some details of the ***fit()*** method). It can sometimes feel safer to make everything explicit. However, remember that writing a custom training loop will make your code longer, more error-prone, and harder to maintain.

> #### **TIP**
> Unless you’re learning or you really need the extra flexibility, you should prefer using the ***fit()*** method rather than implementing your own training loop, especially if you work in a team.

First, let’s build a simple model. There’s no need to compile it, since we will handle the training loop manually:

In [11]:
l2_reg = tf.keras.regularizers.l2(0.05)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30, activation='relu', 
                          kernel_initializer='he_normal', 
                          kernel_regularizer=l2_reg), 
    tf.keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

Next, let’s create a tiny function that will randomly sample a batch of instances from the training set (in Chapter 13 we will discuss the tf.data API, which offers a much better alternative):

In [12]:
import numpy as np

def random_batch(x, y, batch_size=32):
    idx = np.random.randint(len(x), size=batch_size)
    return x[idx], y[idx]

Let’s also define a function that will display the training status, including the number of steps, the total number of steps, the mean loss since the start of the epoch (we will use the ***Mean*** metric to compute it), and other metrics:

In [13]:
def print_status_bar(step, total, loss, metrics=None):
    metrics = ' - '.join([f"{m.name}: {m.result():.4f}"
                          for m in [loss] + (metrics or [])])
    end = '' if step < total else '\n'
    print(f'\r{step}/{total} - ' + metrics, end=end)

This code is self-explanatory, unless you are unfamiliar with Python string formatting: ***{m.result():.4f}*** will format the metric’s result as a float with four digits after the decimal point, and using ***\r*** (carriage return) along with ***end=""*** ensures that the status bar always gets printed on the same line.

With that, let’s get down to business! First, we need to define some hyperparameters and choose the optimizer, the loss function, and the metrics (just the **MAE** in this example):

In [14]:
n_epochs = 5
batch_size = 32
n_steps = len(x_train) // batch_size
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
loss_fn = tf.keras.losses
mean_loss = tf.keras.metrics.Mean(name='mean_loss')
metrics = [tf.keras.metrics.MeanAbsoluteError()]


And now we are ready to build the custom loop!

In [15]:
for epoch in range(1, n_epochs +1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps +1):
        x_batch, y_batch = random_batch(x_train, y_train) # Supposed to be x_train_scalable
        with tf.GradientTape() as tape:
            y_pred = model(x_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
            
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, 
                                      model.trainable_variables))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
            
        print_status_bar(step, n_steps, mean_loss, metrics)
        
    for metric in [mean_loss] + metrics:
        metric.reset_state()

Epoch 1/5


TypeError: 'module' object is not callable

There's a lot going on in this code, so let's walk through it:
- We create two nested loops: one for the epochs, the other for the batches within an epoch.

- Then we sample a random batch from the training set

- Inside the ***tf.GradientTape()*** block, we make a prediction for one batch, using the model as a function, and we compute the loss: it is equal to the main loss plus the other losses (in this model, there is one regularization loss per layer). Since the ***mean_squared_error()*** function returns one loss per instance, we compute the mean over the batch using ***tf.reduce_mean()*** (if you wanted to apply different weights to each instance, this is where you would do it). The regularization losses are already reduced to a single scalar each, so we just need to sum them (using ***tf.add_n()***, which sums multiple tensors of the same shape and data type)

- Next, we ask the tape to compute the gradients of the loss with regard to each trainable variable—not all variables!—and we apply them tothe optimizer to perform a gradient descent step.

- Then we update the mean loss and the metrics (over the current epoch), and we display the status bar.

- At the end of each epoch, we reset the states of the mean loss and the metrics.

If you want to apply gradient clipping (see Chapter 11), set the optimizer’s clipnorm or clipvalue hyperparameter. If you want to apply any other transformation to the gradients, simply do so before calling the apply_gradients() method. And if you want to add weight constraints to your model (e.g., by setting kernel_constraint or bias_constraint when creating a layer), you should update the training loop to apply these constraints just after apply_gradients(), like so:

In [16]:
for variable in model.variables:
    if variable.constraint is not None:
        variable.assign(variable.constraint(variable))

> #### **WARNING**
> Don’t forget to set ***training=True*** when calling the model in the training loop, especially if your model behaves differently during training and testing (e.g., if it uses ***BatchNormalization or Dropout***). If it’s a custom model, make sure to propagate the ***training*** argument to the layers that your model calls.

As you can see, there are quite a lot of things you need to get right, and it’s easy to make a mistake. But on the bright side, you get full control.

Now that you know how to customize any part of your models  and training algorithms, let’s see how you can use TensorFlow’s automatic graph generation feature: it can speed up your custom code considerably, and it will also make it portable to any platform supported by TensorFlow (see Chapter 19).

## **TensorFlow Functions and Graphs**
Back in TensorFlow 1, graphs were unavoidable (as were the complexitiesthat came with them) because they were a central part of TensorFlow’s API.Since TensorFlow 2 (released in 2019), graphs are still there, but not ascentral, and they’re much (much!) simpler to use. To show just how simple,let’s start with a trivial function that computes the cube of its input:

In [2]:
def cube(x):
    return x ** 3

We can obviously call this function with a python value, such as an int or a float, or we can call it with a tensor:

In [3]:
print(cube(2))
print('\n')
print(cube(tf.constant(2.0)))

8


tf.Tensor(8.0, shape=(), dtype=float32)


Now, let's use ***tf.function()*** to convert python function to *TensorFlow Function*:

In [4]:
tf_cube = tf.function(cube)
tf_cube

This TF function can be used exactly like original Python function, and it will return same result(but always as tensors):

In [5]:
print(tf_cube(2))
print('\n')
print(tf_cube(tf.constant(2.0)))

tf.Tensor(8, shape=(), dtype=int32)


tf.Tensor(8.0, shape=(), dtype=float32)


Under the hood, ***tf.function()*** analyzed the computations performedby the ***cube()*** function and generated an equivalent computation graph!As you can see, it was rather painless (we will look at how this worksshortly). Alternatively, we could have used ***tf.function*** as a decorator;this is actually more common:

In [6]:
@tf.function
def tf_cube(x):
    return x ** 3


The original Python function is still available via the TF function’s***python_function*** attribute, in case you ever need it:

In [7]:
tf_cube.python_function(2)

8

TensorFlow optimizes the computation graph, pruning unused nodes,simplifying expressions (e.g., 1 + 2 would get replaced with 3), and more.Once the optimized graph is ready, the TF function efficiently executes theoperations in the graph, in the appropriate order (and in parallel when itcan). As a result, a TF function will usually run much faster than theoriginal Python function, especially if it performs complex computations.Most of the time you will not really need to know more than that: when youwant to boost a Python function, just transform it into a TF function. That’s all!

Moreover, if you set ***jit_compile=True*** when calling ***tf.function()***, then TensorFlow will use *accelerated linear algebra*(**XLA**) to compile dedicated kernels for your graph, often fusing multipleoperations. For example, if your TF function calls ***tf.reduce_sum(a *b + c)***, then without **XLA** the function would first need to compute ***a * b*** and store the result in a temporary variable, then add c to that variable,and lastly call ***tf.reduce_sum()*** on the result. With **XLA**, the wholecomputation gets compiled into a single kernel, which will compute ***tf.reduce_sum(a * b + c)*** in one shot, without using any large temporary variable. Not only will this be much faster, it will also used ramatically less **RAM**.

When you write a custom loss function, a custom metric, a custom layer, or any other custom function and you use it in a Keras model (as we’ve done throughout this chapter), Keras automatically converts your function into a TF function—no need to use ***tf.function()***. So most of the time, the magic is 100% transparent. And if you want Keras to use **XLA**, you just need to set ***jit_compile=True*** when calling the ***compile()*** method. Easy!

> #### **TIP**
> You can tell Keras not to convert your Python functions to TF functions by setting ***dynamic=True*** when creating a custom layer or a custom model. Alternatively, you can set ***run_eagerly=True*** when calling the model’s ***compile()*** method.

By default, a TF function generates a new graph for every unique set of input shapes and data types and caches it for subsequent calls. For example, if you call tf_cube(tf.constant(10)), a graph will be generated for int32 tensors of shape []. Then if you call tf_cube(tf.constant(20)), the same graph will be reused. But if you then call tf_cube(tf.constant([10, 20])), a new graph will be generated for int32 tensors of shape [2]. This is how TF functions handle polymorphism (i.e., varying argument types and shapes). However, this is only true for tensor arguments: if you pass numerical Python values to a TF function, a new graph will be generated for every distinct value: for example, calling tf_cube(10) and tf_cube(20) will generate two graphs.

> #### **WARNING**
> If you call a TF function many times with different numerical Python values, then many graphs will be generated, slowing down your program and using up a lot of RAM (you must delete the TF function to release it). Python values should be reserved for arguments that will have few unique values, such as hyperparameters like the number of neurons per layer. This allows TensorFlow to better optimize each variant of your model.


### **AutoGraph and Tracing**
So how does TensorFlow generate graphs? It starts by analyzing the Python function’s source code to capture all the control flow statements, such as ***for*** loops, ***while*** loops, and ***if*** statements, as well as ***break***, ***continue***, and ***return*** statements. This first step is called *AutoGraph*. The reason TensorFlow has to analyze the source code is that Python does not provide any other way to capture control flow statements: it offers magic methods like ***__add __()*** and ***__mul __()*** to capture operators like ***+*** and * , but there are no ***__while __()*** or ***__if __()*** magic methods. After analyzing the function’s code, AutoGraph outputs an upgraded version of that function in which all the control flow statements are replaced by the appropriate TensorFlow operations, such as ***tf.while_loop()*** for loops and ***tf.cond()*** for if statements.

Next, TensorFlow calls this “upgraded” function, but instead of passing the argument, it passes a *symbolic tensor*—a tensor without any actual value, only a name, a data type, and a shape. For example, if you call ***sum_squares(tf.constant(10))***, then the ***tf__sum_squares()*** function will be called with a symbolic tensor of type int32 and shape []. The function will run in *graph mode*, meaning that each TensorFlow operation will add a node in the graph to represent itself and its output tensor(s) (as opposed to the regular mode, called eager *execution*, or *eager mode*). In graph mode, TF operations do not perform any computations. Graph mode was the default mode in TensorFlow 1.

> #### **TIP**
> In order to view the generated function’s source code, you can call ***tf.autograph.to_code(sum_squares.python_function)***. The code is not meant to be pretty, but it can sometimes help for debugging.

### **TF Function Rules**
Most of the time, converting a Python function that performs TensorFlow operations into a TF function is trivial: decorate it with @tf.function or let Keras take care of it for you. However, there are a few rules to respect:
- If you call any external library, including NumPy or even the standard library, this call will run only during tracing; it will not be part of the graph. Indeed, a TensorFlow graph can only include TensorFlow constructs (tensors, operations, variables, datasets, and so on). So, make sure you use ***tf.reduce_sum()*** instead of ***np.sum()***, ***tf.sort()*** instead of the built-in ***sorted()*** function, and so on (unless you really want the code to run only during tracing). This has a few additional implications:

    - If you define a TF function ***f(x)*** that just returns ***np.random.rand()***, a random number will only be generated when the function is traced, so ***f(tf.constant(2.))*** and ***f(tf.constant(3.))*** will return the same random number, but ***f(tf.constant([2., 3.]))*** will return a different one. If you replace ***np.random.rand()*** with ***tf.random.uniform([])***, then a new random number will be generated upon every call, since the operation will be part of the graph.

    - If your non-TensorFlow code has side effects (such as logging something or updating a Python counter), then you should not expect those side effects to occur every time you call the TF function, as they will only occur when the function is traced.

    - You can wrap arbitrary Python code in a ***tf.py_function()*** operation, but doing so will hinder performance, as TensorFlow will not be able to do any graph optimization on this code. It will also reduce portability, as the graph will only run on platforms where Python is available (and where the right libraries are installed).

- You can call other Python functions or TF functions, but they should follow the same rules, as TensorFlow will capture their operations in the computation graph. Note that these other functions do not need to be decorated with ***@tf.function***.

- If the function creates a TensorFlow variable (or any other stateful TensorFlow object, such as a dataset or a queue), it must do so upon the very first call, and only then, or else you will get an exception. It is usually preferable to create variables outside of the TF function (e.g., in the ***build()*** method of a custom layer). If you want to assign a new value to the variable, make sure you call its ***assign()*** method instead of using the = operator.

- The source code of your Python function should be available to TensorFlow. If the source code is unavailable (for example, if you define your function in the Python shell, which does not give access to the source code, or if you deploy only the compiled **.pyc* Python files to production), then the graph generation process will fail or have limited functionality.

- TensorFlow will only capture ***for*** loops that iterate over a tensor or a ***tf.data.Dataset*** (see Chapter 13). Therefore, make sure you use ***for i in tf.range(x)*** rather than ***for i in range(x)***, or else the loop will not be captured in the graph. Instead, it will run during tracing. (This may be what you want if the ***for*** loop is meant to build the graph; for example, to create each layer in a neural network.)

- As always, for performance reasons, you should prefer a vectorized implementation whenever you can, rather than using loops.

It’s time to sum up! In this chapter we started with a brief overview of TensorFlow, then we looked at TensorFlow’s low-level API, including tensors, operations, variables, and special data structures. We then used these tools to customize almost every component in the Keras API. Finally, we looked at how TF functions can boost performance, how graphs are generated using AutoGraph and tracing, and what rules to follow when you write TF functions (if you would like to open the black box a bit further and explore the generated graphs, you will find technical details in Appendix D).

In the next chapter, we will look at how to eficiently load and preprocess data with TensorFlow.